<a href="https://colab.research.google.com/github/bhaveshgupta01/BCancerGogo/blob/main/GOGO_InceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import StratifiedKFold

# Image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 16

def label_images(directory, target_size=(img_width, img_height)):
    images = []
    labels = []
    class_labels = {'malignant': 0, 'benign': 1, 'normal': 2}

    for class_label, class_index in class_labels.items():
        class_path = os.path.join(directory, class_label)
        for filename in os.listdir(class_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                file_path = os.path.join(class_path, filename)
                image = cv2.imread(file_path)  # You can use PIL.Image.open() as well
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB format
                image = cv2.resize(image, target_size)  # Resize the image to target size
                image = preprocess_input(image)  # Preprocess the image for InceptionV3
                images.append(image)
                labels.append(class_index)

    return np.array(images), np.array(labels)

# Example usage:
directory_path = '/content/drive/MyDrive/MIASdata'
X, y = label_images(directory_path, target_size=(img_width, img_height))

# Split the data into 90% training and 10% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

# Assuming you have three classes (malignant, benign, normal)
num_classes = 3

def build_inception_model(learn_rate=1e-4, momentum=0.9):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

    # Freeze all layers except the last three
    for layer in base_model.layers[:-3]:
        layer.trainable = False

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(layers.Dense(num_classes, activation='softmax'))  # Change activation to 'softmax'

    optimizer = SGD(learning_rate=learn_rate, momentum=momentum)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# ... (your data loading and preprocessing code)
inception_model = build_inception_model()

# Number of folds for k-fold cross-validation
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

for fold, (train_index, val_index) in enumerate(kfold.split(X, y), 1):
    X_train_fold, X_val_fold = X[train_index], X[val_index]
    y_train_fold, y_val_fold = y[train_index], y[val_index]

    # Convert labels to one-hot encoding
    y_train_fold_one_hot = to_categorical(y_train_fold, num_classes=num_classes)
    y_val_fold_one_hot = to_categorical(y_val_fold, num_classes=num_classes)

    # Build a new InceptionV3 model for each fold

    # Train the InceptionV3 model
    class_weights = {0: 4.0, 1: 4.0, 2: 1.0}  # Adjust the weights based on class imbalance

    history = inception_model.fit(
        X_train_fold,
        y_train_fold_one_hot,
        epochs=10,
        validation_data=(X_val_fold, y_val_fold_one_hot),
        batch_size=32,
        verbose=1,
        class_weight=class_weights
    )

    # Evaluate the model on the validation set
    val_loss, val_acc = inception_model.evaluate(X_val_fold, y_val_fold_one_hot, verbose=0)
    print(f"Validation Accuracy for Fold {fold}: {val_acc * 100:.2f}%")


Epoch 1/10
8/8 [==============================] - 50s 6s/step - loss: 3.2153 - accuracy: 0.3008 - val_loss: 1.1408 - val_accuracy: 0.3846
Epoch 2/10
8/8 [==============================] - 40s 5s/step - loss: 2.5711 - accuracy: 0.3867 - val_loss: 1.0848 - val_accuracy: 0.4154
Epoch 3/10
8/8 [==============================] - 43s 6s/step - loss: 1.9575 - accuracy: 0.4570 - val_loss: 1.0632 - val_accuracy: 0.4000
Epoch 4/10
8/8 [==============================] - 40s 5s/step - loss: 1.7665 - accuracy: 0.5117 - val_loss: 1.0552 - val_accuracy: 0.4154
Epoch 5/10
8/8 [==============================] - 43s 6s/step - loss: 1.4964 - accuracy: 0.6094 - val_loss: 1.0592 - val_accuracy: 0.4308
Epoch 6/10
8/8 [==============================] - 40s 5s/step - loss: 1.3285 - accuracy: 0.6094 - val_loss: 1.0658 - val_accuracy: 0.4154
Epoch 7/10
8/8 [==============================] - 43s 6s/step - loss: 1.1672 - accuracy: 0.6758 - val_loss: 1.0728 - val_accuracy: 0.4154
Epoch 8/10
8/8 [==================

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


# Function to calculate and print evaluation metrics
def evaluate_model_multi_class(model, X, y_true):
    # Predictions
    y_pred = model.predict(X)

    # Convert one-hot encoding to class labels
    y_true_labels = np.argmax(y_true, axis=1)
    y_pred_labels = np.argmax(y_pred, axis=1)

    # Calculate accuracy
    accuracy = accuracy_score(y_true_labels, y_pred_labels)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    # Confusion Matrix
    cm = confusion_matrix(y_true_labels, y_pred_labels)
    print("Confusion Matrix:")
    print(cm)

    # Classification Report
    print("Classification Report:")
    print(classification_report(y_true_labels, y_pred_labels))

# Assuming you have trained the model 'inception_model' and loaded the test set 'X_val_fold', 'y_val_fold_one_hot'
evaluate_model_multi_class(inception_model, X_val_fold, y_val_fold_one_hot)


2/2 [==============================] - 5s 239ms/step
Accuracy: 100.00%
Confusion Matrix:
[[ 4  0  0]
 [ 0  9  0]
 [ 0  0 20]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        20

    accuracy                           1.00        33
   macro avg       1.00      1.00      1.00        33
weighted avg       1.00      1.00      1.00        33

